# Based on Research Paper 5

In [ ]:
import pandas as pd

In [ ]:
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [ ]:
#For ISOT
df1=pd.read_csv("/content/drive/MyDrive/Datasets/ISOT/Fake.csv")
df2=pd.read_csv("/content/drive/MyDrive/Datasets/ISOT/True.csv")
article_content=[]
labels=[]
for i in df1['text']:
  article_content.append(i)
  labels.append(0)
for i in df2['text']:
  article_content.append(i)
  labels.append(1)
data={'article_content':article_content,'labels':labels}
df=pd.DataFrame(data)
df.head()

,article_content,labels
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [ ]:
#For FA-KES
df=pd.read_csv("/content/drive/MyDrive/Datasets/FA-KES/FA-KES.csv", encoding='iso-8859-1')
df=df.drop(labels=['unit_id','date','source','location','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10'],axis=1)
df.head()

,article_title,article_content,labels
0,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,0
1,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,0
2,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,0
3,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,0
4,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,0


In [ ]:
def removeURL(text):
  from urllib.parse import urlparse
  lst = [l for l in text.split() if not urlparse(l).scheme]
  s = ' '.join(lst)
  return s

def removeIP(text):
  import re
  pattern = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')
  text = re.sub(pattern, "", text)
  return text

for i in range( df.shape[0] ):
  df['article_content'][i]=removeURL(df['article_content'][i])
  df['article_content'][i]=removeIP(df['article_content'][i])
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.utils import lemmatize

nltk.download('stopwords') # comment out if already downloaded
nltk.download('punkt')     # comment out if already downloaded

df['article_content'] = df['article_content'].apply(lambda x: x.lower())

df.head()

In [ ]:
df['article_content'] = df['article_content'].apply(lambda x:word_tokenize(x))
df.head()

In [ ]:
en_stops = set(stopwords.words('english'))

for i in range( df.shape[0] ):
  filtered=[]
  for word in df['article_content'][i]:
    if word not in en_stops:
      filtered.append(word)
  df['article_content'][i]=filtered

df.head()

In [ ]:
from nltk.stem import PorterStemmer

pStemmer = PorterStemmer()

for i in range( df.shape[0] ):
  filtered=[]
  for word in df['article_content'][i]:
    filtered.append(pStemmer.stem(word))
  df['article_content'][i]=filtered

df.head()

In [ ]:
import gensim
dim=100
w2v_model=gensim.models.Word2Vec(sentences=df['article_content'],size=dim,window=10,min_count=1)



In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()
x=df['article_content']
tokenizer.fit_on_texts(x)
x=tokenizer.texts_to_sequences(x)

maxlen=300
x=pad_sequences(x,maxlen=maxlen)


In [ ]:
vocab_size=len(tokenizer.word_index)+1
vocab=tokenizer.word_index

In [ ]:
import numpy as np
def get_weights(model):
  weight_matrix=np.zeros((vocab_size,dim))

  for word,i in vocab.items():
    weight_matrix[i]=model.wv[word]

  return weight_matrix
embedding_vector=get_weights(w2v_model)



In [ ]:
embedding_vector=get_weights(w2v_model)
embedding_vector.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras import models,datasets,layers,backend
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Conv1D, MaxPool1D, Conv2D, MaxPool2D
from keras.layers.core import Dense, Dropout, Activation, Flatten

In [ ]:
model=models.Sequential()
model.add(Embedding(vocab_size,output_dim=dim,input_length=maxlen,weights=[embedding_vector]))#,weights=[embedding_vector]
model.add( Conv1D(filters=128,kernel_size=5,input_shape=(300,100),activation='relu',padding='valid') )# padding=0,stride=1,filters=128,spatial extent =5
model.add( MaxPool1D( pool_size=2 ) )
model.add( LSTM(units=32) )
model.add( Dense(1,activation='sigmoid') )#sigmoid
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
y=df['labels'].values
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,train_size=0.7,shuffle=False)
x_test, x_val, t_test, y_val = train_test_split(x_test,y_test,test_size = 0.3,train_size =0.7,shuffle=False)

In [ ]:
from sklearn.model_selection import train_test_split
y=df['labels'].values
x_train,x_test,y_train,t_test=train_test_split(x,y,shuffle=True)

In [ ]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_val,y_val),batch_size=64)

Epoch 1/10
7/7 [==============================] - 5s 371ms/step - loss: 0.7113 - acc: 0.5000 - val_loss: 0.6995 - val_acc: 0.5525
Epoch 2/10
7/7 [==============================] - 2s 283ms/step - loss: 0.6896 - acc: 0.5284 - val_loss: 0.7008 - val_acc: 0.4309
Epoch 3/10
7/7 [==============================] - 2s 281ms/step - loss: 0.6788 - acc: 0.5687 - val_loss: 0.6920 - val_acc: 0.5028
Epoch 4/10
7/7 [==============================] - 2s 345ms/step - loss: 0.6632 - acc: 0.6066 - val_loss: 0.6974 - val_acc: 0.5359
Epoch 5/10
7/7 [==============================] - 3s 490ms/step - loss: 0.6514 - acc: 0.6611 - val_loss: 0.6976 - val_acc: 0.5359
Epoch 6/10
7/7 [==============================] - 2s 282ms/step - loss: 0.6340 - acc: 0.6588 - val_loss: 0.7029 - val_acc: 0.5249
Epoch 7/10
7/7 [==============================] - 2s 280ms/step - loss: 0.6297 - acc: 0.6564 - val_loss: 0.7268 - val_acc: 0.4972
Epoch 8/10
7/7 [==============================] - 2s 280ms/step - loss: 0.6164 - acc: 0.64

In [ ]:
y_pred=(model.predict(x_test)>=0.5).astype(int)

7/7 [==============================] - 1s 35ms/step


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(t_test,y_pred)

0.4925373134328358